<a href="https://colab.research.google.com/github/gauravmalaviya143/Pytorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Convert MNIST Image Files into a Tensor of 4-Dimensions(# of images, Height, Width, Color Channels)
transform = transforms.ToTensor()

In [3]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [4]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
#pwd

In [8]:
#ls

In [9]:
#cd ../

In [10]:
#pwd

In [11]:
#ls

In [12]:
#cd cnn_data

In [13]:
#ls

In [14]:
#cd ../

In [15]:
#cd content

In [16]:
#pwd

In [17]:
# Create a small batch size for image...let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [18]:
# Define Our CNN Model
# Describe convolutional layer and ehat it's doing (2 convolutional layers)
conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)

In [19]:
# Grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
    break

In [20]:
X_train.shape

torch.Size([1, 28, 28])

In [21]:
x = X_train.view(1, 1, 28, 28)

In [22]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function

In [23]:
# 1 single image, 6 is the filters we asked for, 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [24]:
# pass through the pooling layer
x = F.max_pool2d(x, 2, 2) # kernal of 2 and stride of 2

In [25]:
x.shape

torch.Size([1, 6, 13, 13])

In [26]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [27]:
x.shape

torch.Size([1, 16, 11, 11])

In [28]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [29]:
x.shape

torch.Size([1, 16, 5, 5])